In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import numpy as np
import branca

In [ ]:
df = pd.read_csv("../Data/CleanData/InitialClean.csv")

df[["DEFUNCION_FETAL","DEFUNCION_NO_FETAL","NACIDO_VIVO"]]=pd.get_dummies(df["RESULTADO_EMB"])

In [ ]:
mortality = df.groupby("COD_DPTO",as_index=False)[["DEFUNCION_FETAL","DEFUNCION_NO_FETAL","NACIDO_VIVO"]].mean().reset_index()
mortality.rename(columns={"DEFUNCION_FETAL":"fetal","DEFUNCION_NO_FETAL":"no_fetal","NACIDO_VIVO":"vivo"},inplace=True)
mortality["deaths"]=mortality["fetal"]+mortality["no_fetal"]
mortality.head(3)

In [ ]:
geo_df = gpd.read_file("../Data/GeoData/Departamentos/MGN_DPTO_POLITICO.shp")

In [ ]:
geo_df.columns

In [ ]:
geo_df["DPTO_CCDGO"] = geo_df["DPTO_CCDGO"].astype(int)

In [ ]:
gdf = (geo_df.set_index('DPTO_CCDGO').join(mortality.set_index('COD_DPTO'))).reset_index()
gdf.head(3)

In [ ]:
#Find centroid of municip. so we can make colored plots in folium
gdf['centroid'] = gdf['geometry'].centroid
gdf['longitude'] = gdf['centroid'].apply(lambda x: np.array(x)[0])
gdf['latitude'] = gdf['centroid'].apply(lambda x: np.array(x)[1])
gdf.drop('centroid',axis=1,inplace=True)

In [ ]:
min_cn, max_cn = gdf["deaths"].quantile([0.01,0.99]).apply(round, 2)

colormap = branca.colormap.LinearColormap(
    colors=['white','green','blue'],
    vmin=min_cn,
    vmax=max_cn
)

colormap.caption="Department mortality"

m = folium.Map(location=[1.2, -73.63])

style_function = lambda x: {
    'fillColor': colormap(x['properties']['deaths']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}

stategeo = folium.GeoJson(
    gdf.to_json(),
    style_function=style_function,  #Controls color filling (which variable, line color, fill color, etc)
    tooltip=folium.GeoJsonTooltip(    #Controls pop-out box when mouse passes over map
        fields=['DPTO_CCDGO', 'deaths'],
        aliases=['ID', 'deaths'],
        localize=True
    )
).add_to(m)

colormap.add_to(m)   #Add the color scale
m

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
#Plot municips. in Antioquia colored by area.
gdf.plot(column='deaths', cmap='Reds', legend=True,ax=ax)

ax.set_title("Muertos fetales o no-fetales por embarazo")
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
#Plot municips. in Antioquia colored by area.
gdf.plot(column='fetal', cmap='Reds', legend=True,ax=ax)

ax.set_title("Muertos fetales por embarazo")
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
#Plot municips. in Antioquia colored by area.
gdf.plot(column='no_fetal', cmap='Reds', legend=True,ax=ax)

ax.set_title("Muertos no-fetales por embarazo")
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
#Plot municips. in Antioquia colored by area.
gdf.plot(column='vivo', cmap='Reds', legend=True,ax=ax)

ax.set_title("Nacimientos por embarazo")
ax.set_xticks([])
ax.set_yticks([])
plt.show()